## Урок 5. SOAP

1. Запросить курсы любой валюты за указанный период. Сохранить полученные данные в базу данных Mongo

In [4]:
from zeep import Client
from pprint import pprint
from pymongo import MongoClient

def get_curs_period(valute, date):
    url='https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL'
    client = Client(url)
    money = client.service.GetCursOnDate(date)
    list_money = money._value_1._value_1
    for item in list_money:
        for v in item.values():
            if v.VchCode == valute:
                print(item['ValuteCursOnDate']['Vcurs'])


valute = 'USD'
date = '2019-08-28'
get_curs_period(valute, date)

Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS


66.2608


2. Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.

3. Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>

4. Доработать приложение по вакансиям. При наличии вакансии с зарплатой отличной от руб. приложение должно осущетсвлять автоматический перевод по курсу валюты на сегодняшний день. Информацию получать через SOAP-сервис центробанка.